# Eventador Brewery Example

For this example we will do some hypothetical brewery automation based on real time sensor data. Sensors on beer mash tuns gather the current temperature levels and produce it to the pipeline. The data is then aggregated in real-time in PipelineDB. This allows the brewer to monitor the temperature levels and ensure it's below a particular threshold for good beer. If it gets out of threshold an actuator can reduce or increase the temperature.

Every notebook has a suite of helper commands that make life easier. These helpers already understand your login creds, endpoints and context. They can be referenced with ```e.help()```

In [ ]:
e.help()

### Inspect the data coming into the pipeline. Let's pull a few messages

In [ ]:
for i in range(1, 10):
    e.kfetch('defaultsink')

### Display the continuous views we have (created as part of the getting started guide)

In [ ]:
display(e.df(e.views))

### Setup the SQL we will be running

In [ ]:
sql = "select * from brewery_sensor_temps"

### Test query the continuous view

In [ ]:
display(e.df(e.query(sql)))

### Tell us if temperatures are out of spec, page the brewmaster. The continuous view is the last 5 minutes of rolling temperature averages.

In [ ]:
import pandas as pd
df = pd.DataFrame(e.query(sql))
df['alert'] = False
for index, row in df.iterrows():
    if row.avg < 59 or row.avg > 62:
       df.set_value(index, 'alert', True)
       ''' perhaps we page the brewmaster
           page_brewmaster(df['sensor'], df['avg'])
       '''
display(df)

### Adjust the temperature using controller API

In [ ]:
target_temp = 60
df = pd.DataFrame(e.query(sql))
for index, row in df.iterrows():
    if row.avg < 59 or row.avg > 62:
       delta = target_temp-row.avg
       ''' throttle the heating equipment
           element.set_target_adjust(delta)
       '''
       print "need adjustment: element by {}".format(delta)

### Watch the data stream in real time by sensor.

In [ ]:
import time
sql = "select * from brewery_sensor_temps"
df = pd.DataFrame(e.query(sql))
display(df)
for i in range(1, 20):
    u = pd.DataFrame(e.query(sql))
    df.update(u)
    clear_output()
    display(df)
    time.sleep(1)

In [ ]:
sql = "select * from brewery_sensor_temps"
df = pd.DataFrame(e.query(sql))
display(df)

### Create a graph of temperatures by device over time